In [32]:
import csv
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

In [37]:
"""

Purpose: This script tries to implement a technique called stacking/blending/stacked generalization.
The reason I have to make this a runnable script because I found that there isn't really any
readable code that demonstrates this technique. You may find the pseudocode in various papers but they
are all each kind of different.

Author: Eric Chio "log0" <im.ckieric@gmail.com>

======================================================================================================
Summary:

Just to test an implementation of stacking. Using a cross-validated random forest and SVMs, I was
only able to achieve an accuracy of about 88% (with 1000 trees and up). Using stacked generalization 
I have seen a maximum of 93.5% accuracy. It does take runs to find it out though. This uses only 
(10, 20, 10) trees for the three classifiers.

This code is heavily inspired from the code shared by Emanuele (https://github.com/emanuele) , but I
have cleaned it up to makeit available for easy download and execution.

======================================================================================================
Methodology:

Three classifiers (RandomForestClassifier, ExtraTreesClassifier and a GradientBoostingClassifier
are built to be stacked by a LogisticRegression in the end.

Some terminologies first, since everyone has their own, I'll define mine to be clear:
- DEV SET, this is to be split into the training and validation data. It will be cross-validated.
- TEST SET, this is the unseen data to validate the generalization error of our final classifier. This
set will never be used to train.

======================================================================================================
Log Output:

X_test.shape = (62L, 6L)
blend_train.shape = (247L, 3L)
blend_test.shape = (62L, 3L)
Training classifier [0]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [1]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [2]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Y_dev.shape = 247
Accuracy = 0.935483870968


"""

class MyStack:
    def __init__(self, *args, **kwargs):
        self.n_trees = 10
        self.n_folds = 5
    
        # Our level 0 classifiers
        self.clfs = [
            RandomForestRegressor(n_estimators = self.n_trees, criterion = 'mse'),
            ExtraTreesRegressor(n_estimators = self.n_trees * 2, criterion = 'mse'),
            GradientBoostingRegressor(n_estimators = self.n_trees),
        ]
        
        # Our level 1 regressor
        self.svr = SVR(kernel='rbf', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1)
        
        

    def train(self, X_dev, Y_dev, X_test):
            
        # Ready for cross validation
        skf = list(StratifiedKFold(Y_dev, self.n_folds))
    
        # Pre-allocate the data
        blend_train = np.zeros((X_dev.shape[0], len(self.clfs))) # Number of training data x Number of classifiers
        blend_test = np.zeros((X_test.shape[0], len(self.clfs))) # Number of testing data x Number of classifiers
    
        #print 'X_test.shape = %s' % (str(X_test.shape))
        #print 'blend_train.shape = %s' % (str(blend_train.shape))
        #print 'blend_test.shape = %s' % (str(blend_test.shape))
    
        # For each classifier, we train the number of fold times (=len(skf))
        for j, clf in enumerate(self.clfs):
            print 'Training classifier [%s]' % (j)
            blend_test_j = np.zeros((X_test.shape[0], len(skf))) # Number of testing data x Number of folds , we will take the mean of the predictions later
            for i, (train_index, cv_index) in enumerate(skf):
                print 'Fold [%s]' % (i)
            
                # This is the training and validation set
                X_train = X_dev[train_index]
                Y_train = Y_dev[train_index]
                X_cv = X_dev[cv_index]
                Y_cv = Y_dev[cv_index]
            
                clf.fit(X_train, Y_train)
            
                # This output will be the basis for our blended classifier to train against,
                # which is also the output of our classifiers
                blend_train[cv_index, j] = clf.predict(X_cv)    
                blend_test_j[:, i] = clf.predict(X_test)
            # Take the mean of the predictions of the cross validation set
            
            blend_test[:, j] = blend_test_j.mean(1)
    
    
        # Start blending!
        self.svr.fit(blend_train, Y_dev)
    
        # Predict now
        Y_test_predict = self.svr.predict(blend_test)
    
        fh = open('predictionsP3.csv','w')    # open file for upload
        fh.write('ID,Prediction\n')         # output header line
        for i,yi in enumerate(Y_test_predict):
          fh.write('{},{}\n'.format(i+1,yi)) # output each prediction
        fh.close()  
    
    
    
        
if __name__ == '__main__':
   
    
    X = np.genfromtxt("Data/kaggle.X1.train.txt",delimiter=',')
    Y = np.genfromtxt("Data/kaggle.Y.train.txt",delimiter=',')
    X_test = np.genfromtxt("Data/kaggle.X1.test.txt",delimiter=',')
    
    X_dev = X[:500,:]
    Y_dev = Y[:500]

    stk = MyStack()
    stk.train(X_dev, Y_dev, X_test)

Training classifier [0]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [1]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
Training classifier [2]
Fold [0]
Fold [1]
Fold [2]
Fold [3]
Fold [4]
